<a href="https://colab.research.google.com/github/GeorgeTelles/world_weekly_report/blob/main/world_weekly_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://postimg.cc/FYtnXpJw">
  <img src="https://i.postimg.cc/t470PmRR/logo-sem-fundo-01.png" alt="logo" width="300">
</a>

---
# **World Weekly Report**
#### Relatório dos principais resultados de indices mundiais da semana
---

# 1. Bibliotecas

In [ ]:
!pip install kaleido

In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime, timedelta
import pytz

import yfinance as yf

import kaleido
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 2. Organizar datas da extração

In [ ]:
#Data de Hoje
hoje = datetime.now()
print(f"Hoje é: {hoje.date()}")

#Traz o dia da Semana
print(f"Hoje é o dia {hoje.weekday()} dessa semana")

#Traz a data de segunda-feira da semana atual
segunda = hoje - timedelta(days = hoje.weekday())
print(f"A segunda feira desta semana foi dia: {segunda.date()}")

#Traz a data de inicio e final da semana
data_inicio = segunda.date().strftime('%Y-%m-%d')
data_final = (hoje.date() + timedelta(days = 2)).strftime('%Y-%m-%d')
print(f"Essa semana vai de {data_inicio} até {data_final}")

In [ ]:
def capa_post_weekly():

    semana_ano = segunda.date().strftime('%U')
    ano = segunda.date().strftime('%Y')

    print('George Telles apresenta')
    print('Weekly Report')
    print(f'Semana {semana_ano} de {ano}')
    print(f'{segunda.date().strftime("%d-%m")} a {(segunda.date() + timedelta(days = 4)).strftime("%d-%m")}')

In [ ]:
capa_post_weekly()

# 3. Índices internacionais

## 3.1. Extração e manipulação dos dados

In [ ]:
dict_yf_world_indices = {
    'Bovespa (Brasil)': ['^BVSP','Brasil'],
    'S&P 500 (EUA)': ['^GSPC', 'EUA'],
    'Dow Jones 30 (EUA)': ['^DJI', 'EUA'],
    'NASDAQ Composite (EUA)': ['^IXIC', 'EUA'],
    'S&P/TSX Composite (Canadá)': ['^GSPTSE', 'Canadá'],
    'S&P/BMV IPC (México)': ['^MXX', 'México'],
    'S&P Merval (Argentina)': ['^MERV', 'Argentina'],
    'Shanghai Composite (China)': ['000001.SS', 'China'],
    'SZSE Component (China)': ['399001.SZ', 'China'],
    'Nikkei 225 (Japão)': ['^N225', 'Japão'],
    'Hang Seng (Hong Kong)': ['^HSI', 'Hong Kong'],
    'BSE Sensex 30 (Índia)': ['^BSESN', 'Índia'],
    'EuroNext 100 (Zona Euro)': ['^N100', 'Zona Euro'],
    'DAX (Alemanha)': ['^GDAXI', 'Alemanha'],
    'FTSE 100 (Reino Unido)': ['^FTSE', 'UK'],
    'CAC 40 (França)': ['^FCHI', 'França'],
    }

df_yf_world_indices = pd.DataFrame.from_dict(dict_yf_world_indices, orient='index', columns=['Ticker_YF', 'Country'])
df_yf_world_indices = df_yf_world_indices.reset_index()
df_yf_world_indices = df_yf_world_indices.rename({'index':'Índice'}, axis=1)
df_yf_world_indices

In [ ]:
world_ohlcv = pd.DataFrame()

for i in df_yf_world_indices.Ticker_YF:

    try:
        ohlcv_indice = pd.DataFrame(yf.download(i, start=data_inicio, end=data_final, progress=False, interval="1d"))
        ohlcv_indice = ohlcv_indice.resample("W").agg({"Open": "first", "High": "max", "Low": "min", "Close": "last"})
        ohlcv_indice["Ticker_YF"] = i
        ohlcv_indice["Resultado_%"] = (ohlcv_indice.Close/ohlcv_indice.Open - 1) * 100
        world_ohlcv = pd.concat([world_ohlcv, ohlcv_indice], axis=0)

    except:
        pass

In [ ]:
world_indices_result = pd.merge(df_yf_world_indices, world_ohlcv, on="Ticker_YF", how="outer")
world_indices_result

## 3.2. Figura dos resultados

In [ ]:
df_fig = world_indices_result.sort_values(by="Resultado_%", ascending=True)
df_fig["Color"] = np.where(df_fig["Resultado_%"] < 0, "red", "green")
df_fig

In [ ]:
annotations = []
x_pos = []
y_pos = []
x_neg = []
y_neg = []

for i in range(len(df_fig)):

    if df_fig["Resultado_%"].iloc[i] > 0:
        y_pos.append(df_fig["Índice"].iloc[i])
        x_pos.append(np.round(df_fig["Resultado_%"].iloc[i], decimals=2))
    else:
        y_neg.append(df_fig["Índice"].iloc[i])
        x_neg.append(np.round(df_fig["Resultado_%"].iloc[i], decimals=2))

eixo_range = max(list(np.abs(x_neg)) + x_pos)*1.25
dist_barra = 2

result_max = df_fig["Resultado_%"].max() + eixo_range
result_min = df_fig["Resultado_%"].min() - eixo_range


# Modelo 1

In [ ]:
fig_indices_results = go.Figure()

fig_indices_results.add_trace(go.Bar(
                        x=df_fig["Resultado_%"],
                        y=df_fig["Índice"],
                        marker_color=df_fig["Color"],
                        orientation="h",
                        )
                    )

for yd, xd in zip(y_pos, x_pos):
    annotations.append(
        dict(
            xref="x1",
            yref="y1",
            y=yd,
            x=xd + dist_barra,
            text=str(xd) + "%",
            font=dict(family="Arial", size=17, color="black"),
            showarrow=False,
        )
    )

for yd, xd in zip(y_neg, x_neg):
    annotations.append(
        dict(
            xref="x1",
            yref="y1",
            y=yd,
            x=xd - dist_barra,
            text=str(xd) + "%",
            font=dict(family="Arial", size=17, color="black"),
            showarrow=False,
        )
    )

fig_indices_results.update_xaxes(
        visible=True,
        showticklabels=True,
        range=[(result_min), (result_max)],
    )

fig_indices_results = fig_indices_results.update_layout(
        title_text=f"<b>Resultado Semanal dos Índices Mundiais</b><br><sub>Resutados de {data_inicio} até {hoje.date()}</sub>",
        title_x=0.5,
        template="simple_white",
        margin=dict(l=20, r=20, t=140, b=20),
        paper_bgcolor="#f7f8fa",
        width=800,
        height=(50 * len(df_fig)),
        annotations=annotations,
        font=dict(family="Arial", size=17, color="black"),
    )

fig_indices_results

#Modelo 2

In [ ]:
fig_indices_results = go.Figure()

# Adiciona o gráfico de barras com cores neon
fig_indices_results.add_trace(go.Bar(
    x=df_fig["Resultado_%"],
    y=df_fig["Índice"],
    marker_color=df_fig["Resultado_%"],
    marker=dict(
        color=df_fig["Resultado_%"],
        colorscale='Viridis',  # Paleta de cores neon
        cmin=df_fig["Resultado_%"].min(),
        cmax=df_fig["Resultado_%"].max(),
        colorbar=None,
    ),
    orientation="h",
    text=df_fig["Resultado_%"].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',  # Ajuste o texto fora da barra
    textfont=dict(family="Arial", size=14, color="white"),
    # Ajuste do espaçamento das barras
    offset=-1
))

fig_indices_results.update_xaxes(
    visible=True,
    showticklabels=True,
    range=[(result_min), (result_max)],
    title_text='Resultado (%)',
    title_font=dict(size=16, family="Arial", color="white"),
    tickfont=dict(size=14, family="Arial", color="white"),
    gridcolor='rgba(255,255,255,0.2)',  # Linhas de grade sutis
)

fig_indices_results.update_yaxes(
    title_text='Índice',
    title_font=dict(size=16, family="Arial", color="white"),
    tickfont=dict(size=14, family="Arial", color="white"),
    gridcolor='rgba(255,255,255,0.2)',  # Linhas de grade sutis
)

fig_indices_results = fig_indices_results.update_layout(
    title=dict(
        text=f"<b>Resultado Semanal dos Índices Mundiais</b><br><sub>Resultados de {data_inicio} até {hoje.date()}</sub>",
        font=dict(size=22, family="Arial", color="white"),
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    title_pad=dict(t=60),
    template="plotly_dark",  # Muda o template para fundo escuro
    margin=dict(l=250, r=30, t=140, b=40),
    paper_bgcolor="#0f0f0f",  # Fundo escuro para um visual neon
    plot_bgcolor="#0f0f0f",  # Fundo do gráfico escuro
    width=900,
    height=(50 * len(df_fig)),
    # annotations=annotations,  # Descomentar se usar anotações manuais
    font=dict(family="Arial", size=16, color="white"),
)

fig_indices_results.show()


#Modelo 3

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig_indices_results = go.Figure()

# Função para definir a cor das barras com base no valor
def get_bar_color(value):
    if value >= 0:
        return 'rgba(46,204,113,1)'  # Verde neon para valores positivos
    else:
        return 'rgba(231,76,60,1)'  # Vermelho neon para valores negativos

# Adiciona o gráfico de barras com cores neon personalizadas
fig_indices_results.add_trace(go.Bar(
    x=df_fig["Resultado_%"],
    y=df_fig["Índice"],
    marker_color=df_fig["Resultado_%"].apply(get_bar_color),
    orientation="h",
    text=df_fig["Resultado_%"].apply(lambda x: f'{x:.2f}%'),
    textposition='outside',  # Ajuste o texto fora da barra
    textfont=dict(family="Arial", size=14, color="white"),
    # Ajuste do espaçamento das barras
    offset=-1
))

# Atualiza os eixos x
fig_indices_results.update_xaxes(
    visible=True,
    showticklabels=True,
    range=[(result_min), (result_max)],
    title_text='Resultado (%)',
    title_font=dict(size=16, family="Arial", color="white"),
    tickfont=dict(size=14, family="Arial", color="white"),
    gridcolor='rgba(255,255,255,0.2)',  # Linhas de grade sutis
)

# Atualiza os eixos y
fig_indices_results.update_yaxes(
    title_text='Índice',
    title_font=dict(size=16, family="Arial", color="white"),
    tickfont=dict(size=14, family="Arial", color="white"),
    gridcolor='rgba(255,255,255,0.2)',
    automargin=True,
)

# Atualiza o layout
fig_indices_results = fig_indices_results.update_layout(
    title=dict(
        text=f"<b>Resultado Semanal dos Índices Mundiais</b><br><sub>Resultados de {data_inicio} até {hoje.date()}</sub>",
        font=dict(size=22, family="Arial", color="white"),
        x=0.5,
        xanchor='center',
        yanchor='top'
    ),
    title_pad=dict(t=60),
    template="plotly_dark",  # Muda o template para fundo escuro
    margin=dict(l=250, r=30, t=140, b=40),
    paper_bgcolor="#0f0f0f",  # Fundo escuro para um visual neon
    plot_bgcolor="#0f0f0f",  # Fundo do gráfico escuro
    width=900,
    height=(50 * len(df_fig)),
    font=dict(family="Arial", size=16, color="white"),
)

fig_indices_results.show()


In [ ]:
fig_indices_results.write_image("fig_internacional.png",  width=800, height=1000, scale=5)

In [ ]:
fig_indices_results.write_html("fig_internacional.html")